In [4]:
import sklearn
import pandas as pd
import pickle
import numpy as np
import pandas_profiling as pp
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
import pickle
from collections import Counter

In [2]:
hg38_annotsv = pd.read_csv('/home/gaziova/CNWeb/AnnotSV.r2yo0QbkMl.tsv', sep='\t', low_memory=False)
df = hg38_annotsv.rename({'AnnotSV ID' : 'AnnotSV_ID', 'SV chrom': 'chrom', 'SV start': 'SV_start', 'SV end': 'SV_end', 'RCV': 'rcvAcc' ,
                          'AnnotSV type': 'AnnotSV_type', 'SV length': 'SV_length', 'SV type': 'SV_type', 'Gene name': 'gene_name'}, axis='columns')
hg38_annotsv.columns
annotsv_split = df[df.AnnotSV_type == 'split']
annotsv_full = df[df.AnnotSV_type == 'full']

In [31]:
hg38_annotsv = hg38_annotsv.rename({'AnnotSV ID' : 'AnnotSV_ID', 'SV chrom': 'chrom', 'SV start': 'SV_start', 'SV end': 'SV_end', 'RCV': 'rcvAcc' ,
                          'AnnotSV type': 'AnnotSV_type', 'SV length': 'SV_length', 'SV type': 'SV_type', 'Gene name': 'gene_name'}, axis='columns')


In [5]:
slovak_CNV = pd.read_csv('/home/gaziova/CNWeb/hg38_publikacia/slovakCNV_AnnotSV.JX2aiSUVe4.tsv', sep='\t')
df = slovak_CNV.rename({'AnnotSV ID' : 'AnnotSV_ID', 'SV chrom': 'chrom', 'SV start': 'SV_start', 'SV end': 'SV_end', 'RCV': 'rcvAcc' ,
                          'AnnotSV type': 'AnnotSV_type', 'SV length': 'SV_length', 'SV type': 'SV_type', 'Gene name': 'gene_name'}, axis='columns')

annotsv_split = df[df.AnnotSV_type == 'split']
annotsv_full = df[df.AnnotSV_type == 'full']

In [4]:
duplikacie = annotsv_full[annotsv_full.SV_type == "DUP"]
delelcie = annotsv_full[annotsv_full.SV_type == 'DEL']

In [3]:
annotsv_full = pd.read_csv('/home/gaziova/CNWeb/hg38_annotated_withDfreq.tsv', sep='\t', low_memory=False)


In [ ]:
#Treba nam tieto:
['CGscore_indel_gene_count_2', 'ExAC_zScore_pLI', 'ExAC_zScore_indel', 'min_D_Frequency', 'gene_count', 'HI_DDDperc_haploinsufficient_gene_count']

In [14]:
 annotsv_split.columns

Index(['AnnotSV_ID', 'chrom', 'SV_start', 'SV_end', 'SV_length', 'SV_type',
       'Unnamed: 0', 'Unnamed: 0.1', 'rcvAcc', 'chr_grch37', 'clinsig',
       'cytoloc', 'data_source', 'disease', 'disease_finding', 'id',
       'link_dbs', 'link_ids', 'observations', 'pubmed', 'start_grch37',
       'stop_grch37', 'title', 'chr_ncbi36', 'start_ncbi36', 'stop_ncbi36',
       'liftOver', 'length', 'AnnotSV_type', 'gene_name', 'NM', 'CDS length',
       'tx length', 'location', 'location2', 'intersectStart', 'intersectEnd',
       'DGV_GAIN_IDs', 'DGV_GAIN_n_samples_with_SV',
       'DGV_GAIN_n_samples_tested', 'DGV_GAIN_Frequency', 'DGV_LOSS_IDs',
       'DGV_LOSS_n_samples_with_SV', 'DGV_LOSS_n_samples_tested',
       'DGV_LOSS_Frequency', 'GD_ID', 'GD_AN', 'GD_N_HET', 'GD_N_HOMALT',
       'GD_AF', 'GD_POPMAX_AF', 'GD_ID_others', '1000g_event', '1000g_AF',
       '1000g_max_AF', 'IMH_ID', 'IMH_AF', 'IMH_ID_others', 'promoters',
       'dbVar_event', 'dbVar_variant', 'dbVar_status', 'GHid_e

In [4]:
def gene_count(gene_names):
    if pd.isnull(gene_names):
        return 0
    return gene_names.count('/') +1

# def promoters_count(gene_names):
#     if pd.isnull(gene_names):
#         return 0
#     return gene_names.count('/') + 1

def y_clinSign(clinsig):  #clinSign_name):
    if clinsig == 'Pathogenic':  #clinSign_name == 'Pathogenic':
        return 1
    else:
        return 0

In [5]:
annotsv_full['gene_count'] = annotsv_full.gene_name.apply(gene_count)
# annotsv_full['promoters_count'] = annotsv_full.promoters.apply(promoters_count)
annotsv_full['clinSign_bin'] = annotsv_full.clinsig.apply(y_clinSign)

/home/gaziova/miniconda3/envs/gaziova_jupyter/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/gaziova/miniconda3/envs/gaziova_jupyter/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
def morbidGenes_count(annotsv_full, morbid_cnv):
    dict_morbidGenes = {}
    for _, row in morbid_cnv.iterrows():
        id_cnv = row.rcvAcc
        if id_cnv not in dict_morbidGenes:
            count = 1
        else:
            count = dict_morbidGenes[id_cnv] + 1
        dict_morbidGenes[id_cnv] = count

    annotsv_full['morbidGene_count'] = annotsv_full.rcvAcc.apply(lambda rcvAcc: dict_morbidGenes.get(rcvAcc, 0))
    return annotsv_full


In [5]:
morbid_cnv = annotsv_split[annotsv_split.morbidGenes == 'yes'] #  || annotsv_split.morbidGenesCandidates == 'yes']

annotsv_full = morbidGenes_count(annotsv_full, morbid_cnv)

In [ ]:
#nemozeme teraz pouzit DDD_frequenciu lebo vo vysledku z annotsv nie je
def D_gene_freq(row):
    D_freq = -1
    if row.SV_type == 'DUP':
        if row.DGV_GAIN_n_samples_with_SV <= 0 or row.DGV_GAIN_n_samples_tested <= 0:
            if row.DDD_DUP_Frequency > 0:
                D_freq = row.DDD_DUP_Frequency
        elif row.DDD_DUP_Frequency <= 0 or row.DDD_DUP_n_samples_with_SV <= 0:
            if row.DGV_GAIN_Frequency > 0:
                D_freq = row.DGV_GAIN_Frequency
        else:
            D_freq = (row.DGV_GAIN_n_samples_with_SV + row.DDD_DUP_n_samples_with_SV) / (
                        row.DGV_GAIN_n_samples_tested + (row.DDD_DUP_Frequency * row.DDD_DUP_n_samples_with_SV))

    else:
        if row.DGV_LOSS_n_samples_with_SV <= 0 or row.DGV_GAIN_n_samples_tested <= 0:
            if row.DDD_DEL_Frequency > 0:
                D_freq = row.DDD_DEL_Frequency
        elif row.DDD_DEL_Frequency <= 0 or row.DDD_DEL_n_samples_with_SV <= 0:
            if row.DGV_LOSS_Frequency > 0:
                D_freq = row.DGV_LOSS_Frequency
        else:
            D_freq = (row.DGV_LOSS_n_samples_with_SV + row.DDD_DEL_n_samples_with_SV) / (
                        row.DGV_LOSS_n_samples_tested + (row.DDD_DEL_Frequency * row.DDD_DEL_n_samples_with_SV))
    return D_freq


In [18]:
#nemozeme teraz pouzit DDD_frequenciu lebo vo vysledku z annotsv nie je
#preto vyberieme len frekvenciu z DGV
def D_gene_freq(row):
    D_freq = -1
    if row.SV_type == 'DUP':
        if row.DGV_GAIN_Frequency > 0:
            D_freq = row.DGV_GAIN_Frequency
        
    else:
        if row.DGV_LOSS_Frequency > 0:
            D_freq = row.DGV_LOSS_Frequency
    
    return D_freq

In [19]:
annotsv_split['min_D_Frequency'] = annotsv_split.apply(D_gene_freq, axis=1)

/home/gaziova/miniconda3/envs/gaziova_jupyter/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [37]:
#nemozeme teraz pouzit DDD_frequenciu lebo vo vysledku z annotsv nie je , 
# preto pocitame frekvenciu iba z DGV(obsahuje zdrave cnv)
def D_gene_freq_2(row):
    D_freq = -1
    if row.SV_type == 'DUP':
        if row.DGV_GAIN_n_samples_with_SV > 0 or row.DGV_GAIN_n_samples_tested > 0:
                #D_freq = row.DDD_DUP_Frequency
                D_freq = round((row.DGV_GAIN_n_samples_with_SV / row.DGV_GAIN_n_samples_tested), 8)
    else:
        if row.DGV_LOSS_n_samples_with_SV > 0 and row.DGV_GAIN_n_samples_tested > 0:
            D_freq = round((row.DGV_LOSS_n_samples_with_SV / row.DGV_LOSS_n_samples_tested), 8)
    
    return D_freq

annotsv_split['min_D_Frequency_2'] = annotsv_split.apply(D_gene_freq_2, axis=1)

/home/gaziova/miniconda3/envs/gaziova_jupyter/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [34]:
#nemozeme teraz pouzit DDD_frequenciu lebo vo vysledku z annotsv nie je , 
# preto pocitame frekvenciu iba z DGV(obsahuje zdrave cnv)
def D_gene_freq_3(row):
    D_freq = -1
    if row.SV_type == 'DUP':
        if row.DGV_GAIN_Frequency > 0: 
            D_freq = row.DGV_GAIN_Frequency
    else:
        if row.DGV_LOSS_Frequency > 0:
            D_freq = row.DGV_LOSS_Frequency
    return D_freq

annotsv_split['min_D_Frequency_3'] = annotsv_split.apply(D_gene_freq_3, axis=1)

/home/gaziova/miniconda3/envs/gaziova_jupyter/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [38]:
x = annotsv_split[['min_D_Frequency_2', 'min_D_Frequency_3']]
c = 0
m = 0
cm = 0
n = 0
for _, row in x.iterrows():
    if np.isnan(row.min_D_Frequency_2) or np.isnan(row.min_D_Frequency_3 < 0):
        n+=1
        print(row.min_D_Frequency_2, row.min_D_Frequency_3)
    else:
        if row.min_D_Frequency_2 != row.min_D_Frequency_3:
            c+=1
#             print(row.min_D_Frequency_2, row.min_D_Frequency_3)
            if row.min_D_Frequency_2 < 0 or row.min_D_Frequency_3 < 0:
                cm +=1
                print(row.min_D_Frequency_2, row.min_D_Frequency_3)
        if row.min_D_Frequency_2 < 0 or row.min_D_Frequency_3 < 0:
            m += 1

-1.0 0.0007554
-1.0 0.0007554
-1.0 0.0007554
-1.0 0.0007554
-1.0 0.0007554
-1.0 0.0007554
-1.0 0.0007554
-1.0 0.0007554
-1.0 0.0007554
-1.0 0.0007554
-1.0 0.00082327
-1.0 0.00082327
-1.0 0.00055356
-1.0 0.00010212
-1.0 0.0007427
-1.0 0.00040437
-1.0 0.00109439
-1.0 0.00287384
-1.0 0.00151362
-1.0 0.00151362
-1.0 0.00287384
-1.0 0.00151362
-1.0 0.00151362
-1.0 0.00151362
-1.0 0.00151362
-1.0 0.00298219
-1.0 0.00024262
-1.0 0.0002553


In [40]:
# minimalna frekvencia pre cnv z master_frekvencii genov
dict_master_freq = {}

for _, row in annotsv_split.iterrows():
    cnv_id = row.rcvAcc
    min_D_freq = row.min_D_Frequency_3
    
    if cnv_id not in dict_master_freq:
        dict_master_freq[cnv_id] = (0, min_D_freq)
    else:
#         prev_master_freq = dict_master_freq[cnv_id][0]
        prev_D_freq = dict_master_freq[cnv_id][1]
#         min_master_freq = min(min_master_freq, prev_master_freq)
        min_D_freq = min(min_D_freq, prev_D_freq)
        dict_master_freq[cnv_id] = (0, min_D_freq)

# annotsv_full['master_cnv_freq_withMinD'] = annotsv_full.rcvAcc.apply(lambda rcvAcc: dict_master_freq.get(rcvAcc, (0, 0))[0])
annotsv_full['min_D_Frequency_3'] = annotsv_full.rcvAcc.apply(lambda rcvAcc: dict_master_freq.get(rcvAcc, (0, 0))[1])
# annotsv_full['master_cnv_freq'] = annotsv_split.apply(master_freq_cnv, axis = 1)

/home/gaziova/miniconda3/envs/gaziova_jupyter/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [39]:
c, cm, m, n

(28, 28, 742, 0)

In [41]:
slovak_CNV = pd.read_csv('/home/gaziova/CNWeb/hg38_publikacia/slovakCNV_hg38_annotated.tsv', sep='\t')


In [51]:
n = 0
for i in range(220):
    if np.isnan(slovak_CNV['min_D_Frequency'].iloc[i]) or np.isnan(annotsv_full['min_D_Frequency_3'].iloc[i]):
        print(slovak_CNV['min_D_Frequency'].iloc[i], annotsv_full['min_D_Frequency_3'].iloc[i])
        n += 1
    else:
        if slovak_CNV['min_D_Frequency'].iloc[i] != annotsv_full['min_D_Frequency_3'].iloc[i]:
            print(slovak_CNV['min_D_Frequency'].iloc[i], annotsv_full['min_D_Frequency_3'].iloc[i])
            

0.0007553999999999999 0.0007554
0.00037769999999999997 0.0003777
0.00037769999999999997 0.0003777
0.00037769999999999997 0.0003777


In [55]:
max(slovak_CNV['min_D_Frequency'])

0.16957306

In [17]:
for _, row in annotsv_split.iterrows():
    if row.DGV_LOSS_Frequency < 0 and row.DGV_LOSS_Frequency != -1:
        print(row.DGV_LOSS_Frequency)

In [20]:
# minimalna frekvencia pre cnv z master_frekvencii genov
dict_master_freq = {}

for _, row in annotsv_split.iterrows():
    cnv_id = row.rcvAcc
    
#     min_master_freq = row.master_gene_freq_withMinD  #row.master_gene_freq
    min_D_freq = row.min_D_Frequency
    
    #if min_master_freq < 0:
        
        #min_master_freq = 1
#     if min_D_freq < 0:
#         min_D_freq = 1
    
    if cnv_id not in dict_master_freq:
        dict_master_freq[cnv_id] = (0, min_D_freq)
    else:
#         prev_master_freq = dict_master_freq[cnv_id][0]
        prev_D_freq = dict_master_freq[cnv_id][1]
#         min_master_freq = min(min_master_freq, prev_master_freq)
        min_D_freq = min(min_D_freq, prev_D_freq)
        dict_master_freq[cnv_id] = (0, min_D_freq)

# annotsv_full['master_cnv_freq_withMinD'] = annotsv_full.rcvAcc.apply(lambda rcvAcc: dict_master_freq.get(rcvAcc, (0, 0))[0])
annotsv_full['min_D_Frequency'] = annotsv_full.rcvAcc.apply(lambda rcvAcc: dict_master_freq.get(rcvAcc, (0, 0))[1])
# annotsv_full['master_cnv_freq'] = annotsv_split.apply(master_freq_cnv, axis = 1)

In [ ]:
annotsv_split['min_D_Frequency'] = annotsv_split.apply(D_gene_freq, axis=1)

In [13]:
def DDD_del_pLI_gene_count(annotsv_split, annotsv_full):
    decifer_dict = {}
    for _, row in annotsv_split.iterrows():
        cnv_id = row.rcvAcc
        haploinsufficiency_perc_decifer = row.HI_DDDpercent
        indel = row.SV_type

        if indel == 'DEL':
            if haploinsufficiency_perc_decifer <= 10:  # pri tomto thresholde a delecii sposobia problem
                if cnv_id not in decifer_dict:
                    count = 1
                else:
                    count = decifer_dict[cnv_id] + 1
                decifer_dict[cnv_id] = count
        else:
            decifer_dict[cnv_id] = 0

    annotsv_full['HI_DDDperc_haploinsufficient_gene_count'] = annotsv_full.rcvAcc.apply(
        lambda rcvAcc: decifer_dict.get(rcvAcc, 0))
    return annotsv_full

In [14]:
# pocet genov, ktore pri ich strate sposobia znefunkcnenie
annotsv_full = DDD_del_pLI_gene_count(annotsv_split, annotsv_full)

/home/gaziova/miniconda3/envs/gaziova_jupyter/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:
duplikacie = annotsv_split[annotsv_split.SV_type == 'DUP']
Counter(duplikacie.TriS_CGscore)

Counter({nan: 483754,
         '0': 25964,
         'Not yet evaluated': 5272,
         '2': 51,
         '1': 354,
         '3': 180,
         '40': 66})

In [10]:
Counter(duplikacie.TriS_CGscore.notna())

Counter({False: 483754, True: 31887})

In [15]:
def clinGen_indel_pLI_gene_count_2(annotsv_split, annotsv_full):
    clinGen_dict = {}
    for _, row in annotsv_split.iterrows():
        cnv_id = row.rcvAcc
        indel = row.SV_type
        if indel == 'DEL':
            score = row.HI_CGscore
        else:
            score = row.TriS_CGscore
        if score == 2 or score == 3:
            if cnv_id not in clinGen_dict:
                count = 1
            else:
                count = clinGen_dict[cnv_id] + 1
            clinGen_dict[cnv_id] = count

    annotsv_full['CGscore_indel_gene_count_2'] = annotsv_full.rcvAcc.apply(lambda rcvAcc: clinGen_dict.get(rcvAcc, 0))
    return annotsv_full

In [16]:
annotsv_full = clinGen_indel_pLI_gene_count_2(annotsv_split, annotsv_full)

/home/gaziova/miniconda3/envs/gaziova_jupyter/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [17]:
def Exac_pLI(annotsv_split, annotsv_full):
    exac_pLI_dict = {}

    for _, row in annotsv_split.iterrows():
        pli = row.pLI_ExAC

        if pli >= 0.9:  # extremly LoF intolerant set of genes
            cnv_id = row.rcvAcc
            if cnv_id not in exac_pLI_dict:
                count = 1
            else:
                count = exac_pLI_dict[cnv_id] + 1
            exac_pLI_dict[cnv_id] = count

    annotsv_full['ExAC_zScore_pLI'] = annotsv_full.rcvAcc.apply(lambda rcvAcc: exac_pLI_dict.get(rcvAcc, 0))
    return annotsv_full

In [18]:
# Counting genes in cnv with high probability that gene is intolerant to loss of function mutation
annotsv_split = Exac_pLI(annotsv_split, annotsv_full)

/home/gaziova/miniconda3/envs/gaziova_jupyter/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [19]:
def Exac_zScore_indel(annotsv_split, annotsv_full):
    exac_indel_dict = {}
    for _, row in annotsv_split.iterrows():
        indel = row.SV_type
        cnv_id = row.rcvAcc

        if indel == 'DEL':
            z_score_indel = row.delZ_ExAC
        else:
            z_score_indel = row.dupZ_ExAC
        z_score_cnv = row.cnvZ_ExAC

        if cnv_id in exac_indel_dict:
            z_score_indel_prev = exac_indel_dict[cnv_id][0]
            z_score_cnv_prev = exac_indel_dict[cnv_id][1]

            if np.isnan(z_score_indel):
                z_score_indel = z_score_indel_prev
            else:
                z_score_indel = max(z_score_indel, z_score_indel_prev)
            if np.isnan(z_score_cnv):
                z_score_cnv = z_score_cnv_prev
            else:
                z_score_cnv = max(z_score_cnv, z_score_cnv_prev)  # min

        if np.isnan(z_score_indel):
            z_score_indel = -10
        if np.isnan(z_score_cnv):
            z_score_cnv = -10
        exac_indel_dict[cnv_id] = (z_score_indel, z_score_cnv)

    annotsv_full['ExAC_zScore_indel'] = annotsv_full.rcvAcc.apply(lambda rcvAcc: exac_indel_dict.get(rcvAcc, (0, 0))[0])
    annotsv_full['ExAc_zScore_cnv'] = annotsv_full.rcvAcc.apply(lambda rcvAcc: exac_indel_dict.get(rcvAcc, (0, 0))[1])
    return annotsv_full

In [20]:
# exac score a Z_score
annotsv_full = Exac_zScore_indel(annotsv_split, annotsv_full)

/home/gaziova/miniconda3/envs/gaziova_jupyter/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/gaziova/miniconda3/envs/gaziova_jupyter/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
['CGscore_indel_gene_count_2', 'ExAC_zScore_pLI', 'ExAC_zScore_indel', 'min_D_Frequency', 'gene_count', 'HI_DDDperc_haploinsufficient_gene_count'] in annotsv_full.columns

In [16]:
delecie = hg38[hg38.type == 'copy number loss']
duplikacie = hg38[hg38.type == 'copy number gain']
Counter(delecie.clinsig), Counter(duplikacie.clinsig)

(Counter({'Pathogenic': 6055,
          'Uncertain significance': 6611,
          'Likely benign': 1742,
          'Benign': 12094,
          'drug response': 18,
          'Conflicting interpretations of pathogenicity': 12,
          'Likely pathogenic': 713,
          'Pathogenic/Likely pathogenic': 6,
          'conflicting data from submitters': 79,
          'Benign/Likely benign': 49,
          'not provided': 32}),
 Counter({'Pathogenic': 2901,
          'Benign': 9608,
          'Uncertain significance': 10859,
          'Likely benign': 2703,
          'not provided': 32,
          'Likely pathogenic': 452,
          'Benign/Likely benign': 146,
          'conflicting data from submitters': 114,
          'Conflicting interpretations of pathogenicity': 19,
          'Pathogenic/Likely pathogenic': 5}))

In [ ]:
# Create file with annotations from annotsv (full a an split records) and new attributes aggregated from them  

In [14]:
# iba ze cnv ma rizikovy gen (0/1) (DDD_del_pLI_gene_count = pocet rizikovych genov)
def DDD_del_pLI_risk(annotsv_split, annotsv_full):
    decifer_dict = {}
    for _, row in annotsv_split.iterrows():
        cnv_id = row.rcvAcc
        haploinsufficiency_perc_decifer = row.HI_DDDpercent
        indel = row.SV_type

        if indel == 'DEL':
            if haploinsufficiency_perc_decifer <= 10:  # pri tomto thresholde a delecii sposobia problem
                if cnv_id not in decifer_dict:
                    decifer_dict[cnv_id] = 1
        else:
            decifer_dict[cnv_id] = np.NaN

    annotsv_full['HI_DDDperc_haploinsufficiecy_risk'] = annotsv_full.rcvAcc.apply(
        lambda rcvAcc: decifer_dict.get(rcvAcc, 0))
    return annotsv_full

annotsv_full = DDD_del_pLI_risk(annotsv_split, annotsv_full)

In [11]:
#nemozeme teraz pouzit DDD_frequenciu lebo vo vysledku z annotsv nie je , 
# preto pocitame frekvenciu iba z DGV(obsahuje zdrave cnv)
def D_gene_freq_max_2(row):
    D_freq = -1
    if row.SV_type == 'DUP':
        if row.DGV_GAIN_n_samples_with_SV > 0 or row.DGV_GAIN_n_samples_tested > 0:
                #D_freq = row.DDD_DUP_Frequency
                D_freq = round((row.DGV_GAIN_n_samples_with_SV / row.DGV_GAIN_n_samples_tested), 8)
    else:
        if row.DGV_LOSS_n_samples_with_SV > 0 and row.DGV_GAIN_n_samples_tested > 0:
            D_freq = round((row.DGV_LOSS_n_samples_with_SV / row.DGV_LOSS_n_samples_tested), 8)
    
    return D_freq

annotsv_split['max_D_Frequency_2'] = annotsv_split.apply(D_gene_freq_max_2, axis=1)

/home/gaziova/miniconda3/envs/gaziova_jupyter/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [13]:
dict_master_freq = {}

for _, row in annotsv_split.iterrows():
    cnv_id = row.rcvAcc
    max_D_freq = row.max_D_Frequency_2

    if cnv_id not in dict_master_freq:
        dict_master_freq[cnv_id] = (0, max_D_freq)
    else:
        prev_D_freq = dict_master_freq[cnv_id][1]
        max_D_freq = max(max_D_freq, prev_D_freq)
        dict_master_freq[cnv_id] = (0, max_D_freq)

annotsv_full['max_D_Frequency_2'] = annotsv_full.rcvAcc.apply(lambda rcvAcc: dict_master_freq.get(rcvAcc, (0, 0))[1])

In [8]:
def D_gene_freq_max(row):
    D_freq = -1
    if row.SV_type == 'DUP':
        if row.DGV_GAIN_Frequency > 0:
            D_freq = row.DGV_GAIN_Frequency
        
    else:
        if row.DGV_LOSS_Frequency > 0:
            D_freq = row.DGV_LOSS_Frequency
    
    return D_freq

annotsv_split['max_D_Frequency'] = annotsv_split.apply(D_gene_freq_max, axis=1)

/home/gaziova/miniconda3/envs/gaziova_jupyter/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [9]:
dict_master_freq = {}

for _, row in annotsv_split.iterrows():
    cnv_id = row.rcvAcc
    max_D_freq = row.max_D_Frequency

    if cnv_id not in dict_master_freq:
        dict_master_freq[cnv_id] = (0, max_D_freq)
    else:
        prev_D_freq = dict_master_freq[cnv_id][1]
        max_D_freq = max(max_D_freq, prev_D_freq)
        dict_master_freq[cnv_id] = (0, max_D_freq)

annotsv_full['max_D_Frequency'] = annotsv_full.rcvAcc.apply(lambda rcvAcc: dict_master_freq.get(rcvAcc, (0, 0))[1])

In [7]:
def clinGen_indel_pLI_risk(annotsv_split, annotsv_full):
    clinGen_dict = {}
    for _, row in annotsv_split.iterrows():
        cnv_id = row.rcvAcc
        indel = row.SV_type
        if indel == 'DEL':
            score = row.HI_CGscore
        else:
            score = row.TriS_CGscore
        if score == 2 or score == 3:
            if cnv_id not in clinGen_dict:
                clinGen_dict[cnv_id] = 1

    annotsv_full['CGscore_indel_risk'] = annotsv_full.rcvAcc.apply(lambda rcvAcc: clinGen_dict.get(rcvAcc, 0))
    return annotsv_full

annotsv_full = clinGen_indel_pLI_risk(annotsv_split, annotsv_full)

In [6]:
def morbidGenes_risk(annotsv_full, morbid_cnv):
    dict_morbidGenes = {}
    for _, row in morbid_cnv.iterrows():
        cnv_id = row.rcvAcc
        if cnv_id not in dict_morbidGenes:
            dict_morbidGenes[cnv_id] = 1

    annotsv_full['morbidGene_risk'] = annotsv_full.rcvAcc.apply(lambda rcvAcc: dict_morbidGenes.get(rcvAcc, 0))
    return annotsv_full

morbid_cnv = annotsv_split[annotsv_split.morbidGenes == 'yes'] #  || annotsv_split.morbidGenesCandidates == 'yes']
annotsv_full = morbidGenes_risk(annotsv_full, morbid_cnv)

In [5]:
def Exac_pLI_risk(annotsv_split, annotsv_full):
    exac_pLI_dict = {}

    for _, row in annotsv_split.iterrows():
        pli = row.pLI_ExAC

        if pli >= 0.9:  # extremly LoF intolerant set of genes
            cnv_id = row.rcvAcc
            if cnv_id not in exac_pLI_dict:
                exac_pLI_dict[cnv_id] = 1

    annotsv_full['ExAC_zScore_pLI_risk'] = annotsv_full.rcvAcc.apply(lambda rcvAcc: exac_pLI_dict.get(rcvAcc, 0))
    return annotsv_full

annotsv_full = Exac_pLI_risk(annotsv_split, annotsv_full)

In [4]:
def Exac_zScore_indel_min(annotsv_split, annotsv_full):
    exac_indel_dict = {}
    for _, row in annotsv_split.iterrows():
        indel = row.SV_type
        cnv_id = row.rcvAcc

        if indel == 'DEL':
            z_score_indel = row.delZ_ExAC
        else:
            z_score_indel = row.dupZ_ExAC
        z_score_cnv = row.cnvZ_ExAC

        if cnv_id in exac_indel_dict:
            z_score_indel_prev = exac_indel_dict[cnv_id][0]
            z_score_cnv_prev = exac_indel_dict[cnv_id][1]

            if np.isnan(z_score_indel):
                z_score_indel = z_score_indel_prev
            else:
                z_score_indel = min(z_score_indel, z_score_indel_prev)
            if np.isnan(z_score_cnv):
                z_score_cnv = z_score_cnv_prev
            else:
                z_score_cnv = min(z_score_cnv, z_score_cnv_prev)

        if np.isnan(z_score_indel):
            z_score_indel = -10
        if np.isnan(z_score_cnv):
            z_score_cnv = -10
        exac_indel_dict[cnv_id] = (z_score_indel, z_score_cnv)

    annotsv_full['ExAC_zScore_indel_min'] = annotsv_full.rcvAcc.apply(lambda rcvAcc: exac_indel_dict.get(rcvAcc, (0, 0))[0])
    annotsv_full['ExAc_zScore_cnv'] = annotsv_full.rcvAcc.apply(lambda rcvAcc: exac_indel_dict.get(rcvAcc, (0, 0))[1])
    return annotsv_full

annotsv_full = Exac_zScore_indel_min(annotsv_split, annotsv_full)

In [ ]:
# maxima, minima, pocty ulozit tu
# ine pocty a nove stributy do generate_features

def gene_count(gene_names):
    if pd.isnull(gene_names):
        return 0
    return gene_names.count('/') +1

def promoters_count(gene_names):
    if pd.isnull(gene_names):
        return 0
    return gene_names.count('/') + 1

def y_clinSign(clinSign_name):
    if clinSign_name == 'Pathogenic':
        return 1
    else:
        return 0

# frekvencia genu z DDD a DGV do jednej min_D_Frequency
def D_gene_freq(row):
    D_freq = -1
    if row.SV_type == 'DUP':
        if row.DGV_GAIN_n_samples_with_SV <= 0 or row.DGV_GAIN_n_samples_tested <= 0:
            if row.DDD_DUP_Frequency > 0:
                D_freq = row.DDD_DUP_Frequency
        elif row.DDD_DUP_Frequency <= 0 or row.DDD_DUP_n_samples_with_SV <= 0:
            if row.DGV_GAIN_Frequency > 0:
                D_freq = row.DGV_GAIN_Frequency
        else:
            D_freq = (row.DGV_GAIN_n_samples_with_SV + row.DDD_DUP_n_samples_with_SV) / (
                        row.DGV_GAIN_n_samples_tested + (row.DDD_DUP_Frequency * row.DDD_DUP_n_samples_with_SV))

    else:
        if row.DGV_LOSS_n_samples_with_SV <= 0 or row.DGV_GAIN_n_samples_tested <= 0:
            if row.DDD_DEL_Frequency > 0:
                D_freq = row.DDD_DEL_Frequency
        elif row.DDD_DEL_Frequency <= 0 or row.DDD_DEL_n_samples_with_SV <= 0:
            if row.DGV_LOSS_Frequency > 0:
                D_freq = row.DGV_LOSS_Frequency
        else:
            D_freq = (row.DGV_LOSS_n_samples_with_SV + row.DDD_DEL_n_samples_with_SV) / (
                        row.DGV_LOSS_n_samples_tested + (row.DDD_DEL_Frequency * row.DDD_DEL_n_samples_with_SV))
    return D_freq

def master_freq_gene_withMinD(row, db_frequencies):
    min_db_freq = 1
    for col in db_frequencies:
        col = str(col)
        if row[col] > 0:
            min_db_freq = min(min_db_freq, row[col])
    return min_db_freq

def morbidGenes_count(annotsv_full, morbid_cnv):
    dict_morbidGenes = {}
    for _, row in morbid_cnv.iterrows():
        id_cnv = row.rcvAcc
        if id_cnv not in dict_morbidGenes:
            count = 1
        else:
            count = dict_morbidGenes[id_cnv] + 1
        dict_morbidGenes[id_cnv] = count

    annotsv_full['morbidGene_count'] = annotsv_full.rcvAcc.apply(lambda rcvAcc: dict_morbidGenes.get(rcvAcc, 0))
    return annotsv_full

def master_freq(annotsv_split, annotsv_full):
    dict_master_freq = {}
    for _, row in annotsv_split.iterrows():
        cnv_id = row.rcvAcc

        min_master_freq = row.master_gene_freq_withMinD  # row.master_gene_freq
        min_D_freq = row.min_D_Frequency

        if cnv_id not in dict_master_freq:
            dict_master_freq[cnv_id] = (min_master_freq, min_D_freq)
        else:
            prev_master_freq = dict_master_freq[cnv_id][0]
            prev_D_freq = dict_master_freq[cnv_id][1]
            min_master_freq = min(min_master_freq, prev_master_freq)
            min_D_freq = min(min_D_freq, prev_D_freq)
            dict_master_freq[cnv_id] = (min_master_freq, min_D_freq)
    annotsv_full['master_cnv_freq_withMinD'] = annotsv_full.rcvAcc.apply(
        lambda rcvAcc: dict_master_freq.get(rcvAcc, (0, 0))[0])
    annotsv_full['min_D_Frequency'] = annotsv_full.rcvAcc.apply(lambda rcvAcc: dict_master_freq.get(rcvAcc, (0, 0))[1])
    return  annotsv_full

def Exac_zScore_indel(annotsv_split, annotsv_full):
    exac_indel_dict = {}
    for _, row in annotsv_split.iterrows():
        indel = row.SV_type
        cnv_id = row.rcvAcc

        if indel == 'DEL':
            z_score_indel = row.delZ_ExAC
        else:
            z_score_indel = row.dupZ_ExAC
        z_score_cnv = row.cnvZ_ExAC

        if cnv_id in exac_indel_dict:
            z_score_indel_prev = exac_indel_dict[cnv_id][0]
            z_score_cnv_prev = exac_indel_dict[cnv_id][1]

            if np.isnan(z_score_indel):
                z_score_indel = z_score_indel_prev
            else:
                z_score_indel = max(z_score_indel, z_score_indel_prev)
            if np.isnan(z_score_cnv):
                z_score_cnv = z_score_cnv_prev
            else:
                z_score_cnv = max(z_score_cnv, z_score_cnv_prev)  # min

        if np.isnan(z_score_indel):
            z_score_indel = -10
        if np.isnan(z_score_cnv):
            z_score_cnv = -10
        exac_indel_dict[cnv_id] = (z_score_indel, z_score_cnv)

    annotsv_full['ExAC_zScore_indel'] = annotsv_full.rcvAcc.apply(lambda rcvAcc: exac_indel_dict.get(rcvAcc, (0, 0))[0])
    annotsv_full['ExAc_zScore_cnv'] = annotsv_full.rcvAcc.apply(lambda rcvAcc: exac_indel_dict.get(rcvAcc, (0, 0))[1])
    return annotsv_full

def Exac_pLI(annotsv_split, annotsv_full):
    exac_pLI_dict = {}

    for _, row in annotsv_split.iterrows():
        pli = row.pLI_ExAC

        if pli >= 0.9:  # extremly LoF intolerant set of genes
            cnv_id = row.rcvAcc
            if cnv_id not in exac_pLI_dict:
                count = 1
            else:
                count = exac_pLI_dict[cnv_id] + 1
            exac_pLI_dict[cnv_id] = count

    annotsv_full['ExAC_zScore_pLI'] = annotsv_full.rcvAcc.apply(lambda rcvAcc: exac_pLI_dict.get(rcvAcc, 0))
    return annotsv_full

def DDD_del_pLI_gene_count(annotsv_split, annotsv_full):
    decifer_dict = {}
    for _, row in annotsv_split.iterrows():
        cnv_id = row.rcvAcc
        haploinsufficiency_perc_decifer = row.HI_DDDpercent
        indel = row.SV_type

        if indel == 'DEL':
            if haploinsufficiency_perc_decifer <= 10:  # pri tomto thresholde a delecii sposobia problem
                if cnv_id not in decifer_dict:
                    count = 1
                else:
                    count = decifer_dict[cnv_id] + 1
                decifer_dict[cnv_id] = count
        else:
            decifer_dict[cnv_id] = 0

    annotsv_full['HI_DDDperc_haploinsufficient_gene_count'] = annotsv_full.rcvAcc.apply(
        lambda rcvAcc: decifer_dict.get(rcvAcc, 0))
    return annotsv_full

def clinGen_indel_pLI_gene_count(annotsv_split, annotsv_full):
    clinGen_dict = {}
    for _, row in annotsv_split.iterrows():
        cnv_id = row.rcvAcc
        indel = row.SV_type
        if indel == 'DEL':
            score = row.HI_CGscore
        else:
            score = row.TriS_CGscore
        if score == 2 or score == 3:
            if cnv_id not in clinGen_dict:
                count = 1
            else:
                count = clinGen_dict[cnv_id] + 1
            clinGen_dict[cnv_id] = count

    annotsv_full['CGscore_indel_gene_count'] = annotsv_full.rcvAcc.apply(lambda rcvAcc: clinGen_dict.get(rcvAcc, 0))
    return annotsv_full

def clinGen_indel_pLI_gene_count_2(annotsv_split, annotsv_full):
    clinGen_dict = {}
    for _, row in annotsv_split.iterrows():
        cnv_id = row.rcvAcc
        indel = row.SV_type
        if indel == 'DEL':
            score = row.HI_CGscore
        else:
            score = row.TriS_CGscore
        if score == 2 or score == 3:
            if cnv_id not in clinGen_dict:
                count = 1
            else:
                count = clinGen_dict[cnv_id] + 1
            clinGen_dict[cnv_id] = count

    annotsv_full['CGscore_indel_gene_count_2'] = annotsv_full.rcvAcc.apply(lambda rcvAcc: clinGen_dict.get(rcvAcc, 0))
    return annotsv_full

    

In [ ]:
#natahat tabulku:
annotsv_file = '../isv_annotsv/data_annotsv/AnnotSV.r2yo0QbkMl.tsv'
annotsv = pd.read_csv(annotsv_file, sep='\t', low_memory=False)
    
annotsv = annotsv.rename({'SV chrom': 'chrom','SV start':'SV_start', 'SV end':'SV_end', 'SV type': 'SV_type',
                'SV length': 'SV_length', 'Gene name': 'gene_name', 'AnnotSV type': 'AnnotSV_type', 
                'RCV': 'rcvAcc'}, axis='columns')

annotsv_full = annotsv[annotsv.AnnotSV_type == 'full']
annotsv_split = annotsv[annotsv.AnnotSV_type == 'split']

annotsv_full['gene_count'] = annotsv_full.gene_name.apply(gene_count)
annotsv_full['promoters_count'] = annotsv_full.promoters.apply(promoters_count)
annotsv_full['clinsig_bin'] = annotsv_full.clinsig.apply(y_clinSign)
annotsv_full['length'] = annotsv_full.SV_end - annotsv_full.SV_start + 1
morbid_cnv = annotsv_split[annotsv_split.morbidGenes == 'yes']  # || annotsv_split.morbidGenesCandidates == 'yes']
annotsv_full = morbidGenes_count(annotsv_full, morbid_cnv)  #annotsv_full['morbidGenes_count'] = annotsv_full..*apply(morbidGenes_count(morbid_cnv))


In [ ]:
# annotsv_split['min_D_Frequency'] = annotsv_split.apply(D_gene_freq, axis=1)

# exac score a Z_score
annotsv_full = Exac_zScore_indel(annotsv_split, annotsv_full)

# Counting genes in cnv with high probability that gene is intolerant to loss of function mutation
annotsv_split = Exac_pLI(annotsv_split, annotsv_full)

# pocet genov, ktore pri ich strate sposobia znefunkcnenie
annotsv_split = DDD_del_pLI_gene_count(annotsv_split, annotsv_full)

# haploinsufficiency & triplosensitivity z HI_cgScore
annotsv_full = clinGen_indel_pLI_gene_count(annotsv_split, annotsv_full)

# haploinsufficiency & triplosensitivity z HI_cgScore
annotsv_full = clinGen_indel_pLI_gene_count_2(annotsv_split, annotsv_full)

# nakoniec ulozit do noveho suboru, s ktorym budeme potom pracovat
# annotsv_full_file = '../isv_annotsv/data_annotsv/AnnotSV.r2yo0QbkMl_full.tsv'
# annotsv_full.to_csv(annotsv_full_file, sep='\t', index=False)


In [ ]:
annotsv_full_file = '../isv_annotsv/data_annotsv/AnnotSV.r2yo0QbkMl_full.tsv'
annotsv_full.to_csv(annotsv_full_file, sep='\t', index=False)